# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [81]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [82]:
# 1) Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

r = requests.get(url)
r.status_code
data_json = r.json()
data_json
pd.DataFrame(data_json['items']).head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10108,Município,São Pedro da Aldeia,3305208,RJ,None,Arquivado,17944.001122/2008-21,2009-03-06T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,5.989000e+05,0,0,13/03/2009
1,56130,Município,Piraí,3304003,RJ,PVL02.001201/2022-28,Encaminhado à PGFN com manifestação técnica fa...,17944.101368/2022-51,2022-06-09T15:47:52Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3.000000e+06,0,0,27/06/2022
2,6266,Município,Maricá,3302700,RJ,None,Regularizado,17944.000308/2010-88,2011-12-19T02:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Ampla Energia e Serviços S/A,Real,8.623476e+05,0,0,20/01/2012
3,34031,Município,Itatiaia,3302254,RJ,PVL02.003023/2019-74,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,5.000000e+06,0,1,17/10/2019
4,30348,Município,Rio de Janeiro,3304557,RJ,PVL02.002745/2018-21,Deferido,17944.109791/2018-12,2018-11-29T11:59:21Z,Operação contratual interna,Aporte de capital,Instituição Financeira Nacional,Banco Santander (Brasil) S.A.,Real,3.000000e+08,1,0,29/11/2018


In [83]:
# 2) Quais são os três status das solicitações mais frequentes na base? Quais são suas frequências?
df = pd.DataFrame(data_json['items'])
df['status'].value_counts()

Deferido                                                 106
Arquivado                                                 53
Regularizado                                              26
Deferido (PVL-IF)                                         24
Arquivado por decurso de prazo                            24
Encaminhado à PGFN com manifestação técnica favorável     14
Arquivado a pedido                                         8
Em retificação pelo interessado                            3
Arquivado a pedido (PVL-IF)                                1
Arquivado pela STN                                         1
Name: status, dtype: int64

In [84]:
# 3) Construa uma nova variável que contenha o ano do status. Observe que data_status vem como tipo object no DataFrame. Dica: você pode usar o método .str para transformar o tipo da variável em string, em seguida um método como slice() ou split().
df['data_status'] = df['data_status'].astype(str)
df['ano'] = df['data_status'].str.split('/').str[-1]
df['ano'].head()

0    2009
1    2022
2    2012
3    2019
4    2018
Name: ano, dtype: object

In [85]:
# 4) Indique a frequência de cada ano do campo construído no item (3).
df['ano'].value_counts()


2008    37
2007    29
2023    28
2014    20
2010    19
2019    16
2020    15
2013    15
2012    13
2022    11
2011    10
2009     9
2015     8
2021     5
2017     5
2006     4
2002     4
2016     4
2018     4
2024     2
2003     1
2004     1
Name: ano, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [94]:
#1) Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado ('Estado'ou Município), e que devolve os dados da consulta no formato DataFrame.

import pandas as pd
import requests

def consultar_dados(UF, tipo_interessado):
    # Verifica se a informação inserida 'estado' ou 'municipio' estão corretos
    if tipo_interessado not in ['Estado', 'Município']:
        print("Tipo de interessado inválido. Escolha 'Estado' ou 'Município'.")
        return None
    
    # Url para consulta a api
    url_api = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
    
    # Pegar api e formartar dados de UF e tipo_interessado
    url_api = url_api.format(UF=UF, tipo_interessado=tipo_interessado)
    
    params = {
        'uf': UF,
        'tipo_interessado': tipo_interessado
    }
    
    # Fazer a requisição à API
    try:
        response = requests.get(url_api, params)
        response.raise_for_status()  # Informa qual o erro 
    except requests.exceptions.RequestException as e:
        print("Erro ao fazer a requisição à API:", e)
        return None
    
    # Verificar se a resposta contém dados válidos
    if response.status_code == 200:
        # Converter os dados da resposta para um DataFrame pandas
        dados = response.json()  # Supondo que a API retorna dados no formato JSON
        df = pd.DataFrame(dados['items'])
        return df
    else:
        print("Falha ao obter os dados da API. Código de status:", response.status_code)
        return None

In [95]:
# 2) Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo' estão registradas?

url_mg = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado'
r_mg = requests.get(url)
r_mg.status_code
data_json_mg = r_mg.json()
data_json_mg
df_mg = pd.DataFrame(data_json_mg['items'])
df_mg['status'].value_counts()['Arquivado por decurso de prazo']
df_ba['status'].value_counts()['Arquivado por decurso de prazo']

136

In [96]:
# 3) Qual é o município da Bahia com mais solicitações deferidas?
url_ba_muni = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio'
r_ba = requests.get(url_ba_muni)
r_ba.status_code
data_json_ba = r_ba.json()
data_json_ba
df_ba = pd.DataFrame(data_json_ba['items'])
df_ba_deferido = df_ba[df_ba['status'] == 'Deferido']
contagem_deferidos_ba = df_ba_deferido['interessado'].value_counts()
municipio_mais_deferido = contagem_deferidos_ba.idxmax()
municipio_mais_deferido

'Luís Eduardo Magalhães'

In [97]:
# 4) Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'
url_ba_estado = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Estado'
r_ba_estado = requests.get(url_ba_estado)
r_ba_estado.status_code
data_json_ba_estado = r_ba_estado.json()
data_json_ba_estado
df_ba_estado = pd.DataFrame(data_json_ba_estado['items'])
df_ba_estado.head()
df_ba_estado.to_csv('DataFrame_Bahia_estado.csv')

In [98]:
df_teste = consultar_dados('BA', 'Estado')
df_teste.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,18368,Estado,Bahia,29,BA,None,Deferido,19407.000008/2005-44,2005-06-30T03:00:00Z,Operação contratual interna,Segurança pública,Instituição Financeira Nacional,Banco do Brasil S/A,Dólar dos EUA,53839498.74,1,0,26/07/2005
1,18389,Estado,Bahia,29,BA,None,Deferido,19407.000013/2002-12,2002-05-31T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,59400000.00,1,0,18/06/2002
2,18511,Estado,Bahia,29,BA,None,Deferido,19407.000039/2002-52,2002-06-11T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2307420.42,1,0,08/07/2002
3,18543,Estado,Bahia,29,BA,None,Deferido,19407.000047/2002-07,2002-06-21T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1481000.00,1,0,03/07/2002
4,10198,Estado,Bahia,29,BA,None,Deferido,17944.001142/2010-17,2010-08-03T03:00:00Z,Operação contratual interna,PMAE,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,9905400.00,1,0,16/08/2010
